In [1]:
from pyscf import gto,scf
import numpy as np,numpy
import matplotlib.pyplot as plt
from FcMole import *

In [2]:
CO=gto.M(atom="C 0 0 0; O 0 0 2 ",unit="Bohr", basis="sto-3g")

In [3]:
%%time
mf=scf.RHF(CO)
mf.scf()

converged SCF energy = -111.199724042754
CPU times: user 2.03 s, sys: 2.09 ms, total: 2.03 s
Wall time: 90.4 ms


-111.19972404275447

In [4]:
mf.e_tot

-111.19972404275447

In [5]:
summary=dict(mf.scf_summary)
summary

{'e1': -199.5102395828109, 'e2': 64.31051554005643, 'nuc': 24.0}

In [6]:
sum(summary.values()) ,mf.e_tot

(-111.19972404275447, -111.19972404275447)

In [7]:
#P=CC^T
C=mf.mo_coeff
Pc=2*C[:,:7].dot(C.T[:7,:])
P0=mf.make_rdm1()

In [8]:
J,K=mf.get_jk()
h1=mf.get_hcore()
F=mf.get_fock()

In [9]:
summary["e1"],np.einsum("ij,ij",h1,P0)

(-199.5102395828109, -199.5102395828109)

In [10]:
summary["e2"],np.einsum("ij,ij",J,P0)/2-np.einsum("ij,ij",K,P0)/4  # it is the HF Coulomb operator 

(64.31051554005643, 64.31051554005643)

In [11]:
np.allclose(F,h1+J-K/2) #F=h1+J-K/2

True

In [12]:
np.einsum("ij,ij",F,P0),summary["e1"]+summary["e2"]*2

(-70.88920850269808, -70.88920850269804)

In [13]:
from functools import reduce
def gen_vind(mf, mo_coeff, mo_occ):
    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    nocc = mocc.shape[1]
    vresp = mf.gen_response(mo_coeff, mo_occ, hermi=1)
    def fx(mo1):
        mo1 = mo1.reshape(-1,nmo,nocc)
        nset = len(mo1)
        dm1 = numpy.empty((nset,nao,nao))
        for i, x in enumerate(mo1):
            dm = reduce(numpy.dot, (mo_coeff, x*2, mocc.T)) # *2 for double occupancy
            dm1[i] = dm + dm.T
        v1 = vresp(dm1)
        v1vo = numpy.empty_like(mo1)
        for i, x in enumerate(v1):
            v1vo[i] = reduce(numpy.dot, (mo_coeff.T, x, mocc))
        return v1vo
    return fx

In [14]:
from pyscf.scf import cphf
from pyscf.scf import  _response_functions

In [15]:
vind=gen_vind(mf, mf.mo_coeff, mf.mo_occ)

In [16]:
vind(np.random.rand(10,7)).shape

(1, 10, 7)

In [17]:
mf.mo_occ

array([2., 2., 2., 2., 2., 2., 2., 0., 0., 0.])

In [19]:
rf=mf.gen_response(mf.mo_coeff, mf.mo_occ, hermi=1)
dm1test=np.ones((10,10))/100
rf(dm1test)

array([[ 0.08849428,  0.01495866, -0.00524675, -0.00524675, -0.00405709,
        -0.00422077, -0.00175978, -0.00392383, -0.00392383, -0.00688321],
       [ 0.01495866,  0.05948812, -0.00139476, -0.00139476,  0.00572699,
        -0.00021311,  0.02728125, -0.0032178 , -0.0032178 , -0.02931106],
       [-0.00524675, -0.00139476,  0.06038057, -0.00382447, -0.00338306,
        -0.00433104, -0.00509075,  0.01693565, -0.00359605, -0.00089858],
       [-0.00524675, -0.00139476, -0.00382447,  0.06038057, -0.00338306,
        -0.00433104, -0.00509075, -0.00359605,  0.01693565, -0.00089858],
       [-0.00405709,  0.00572699, -0.00338306, -0.00338306,  0.0648409 ,
         0.0035425 ,  0.03513864, -0.00255662, -0.00255662, -0.02870674],
       [-0.00422077, -0.00021311, -0.00433104, -0.00433104,  0.0035425 ,
         0.12396744,  0.01332848, -0.00749146, -0.00749146, -0.00320253],
       [-0.00175978,  0.02728125, -0.00509075, -0.00509075,  0.03513864,
         0.01332848,  0.06925861, -0.00416837

In [24]:
vj, vk = mf.get_jk(CO, dm1test, hermi=1)
respfun=rf(dm1test)
np.allclose(vj - .5 * vk,rf(dm1test))

True

In [31]:
dFt=mf.get_fock(dm=P0+dm1test)-mf.get_fock(dm=P0)

In [33]:
np.allclose(dFt,respfun)

True